In [ ]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from datetime import date, timedelta
import os

import neuralNets

def oneHotEncodeData3Classes(targets):
    j=0
    Y_val = np.zeros((targets.shape[0], 3))
    for j in range(targets.shape[0]):
        if targets[j] == 0:
            Y_val[j, 0] = 1
        elif targets[j] == 1:
            Y_val[j, 1] = 1
        elif targets[j] == 2:
            Y_val[j, 2] = 1
        else:
            print("something went wrong, new class", targets[j])
    return Y_val

In [ ]:
tic = time.time()
#training_data = pd.read_csv("appendedTrainingData_test12021-01-31_2021-03-21.csv")
training_data = pd.read_csv("data/numerai_datasets_02.05.21/numerai_training_data.csv")
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

training_data[feature_cols] = training_data[feature_cols].astype(np.float16)
training_data.target        = training_data.target.astype(np.float16)

# X_train = training_data[feature_cols].to_numpy()
# Y_train = training_data.target.to_numpy()

In [ ]:
validation_data = pd.read_csv("data/numerai_datasets_02.05.21/numerai_validation_data.csv")  

In [ ]:
# X_train_biased = training_data[feature_cols] - 0.5
# X_val_biased = validation_data[feature_cols] - 0.5

# train classifier between 0/1 0.25/0.75 and 0.5

In [ ]:
X_train_3class = training_data[feature_cols]
#X_train_3class = X_train_biased
Y_train_3class = training_data.target

Y_train_3class = Y_train_3class.replace(1, 0)
Y_train_3class = Y_train_3class.replace([0.25, 0.75], 1)
Y_train_3class = Y_train_3class.replace(0.5, 2)

X_train_3class = X_train_3class.to_numpy()
Y_train_3class = Y_train_3class.to_numpy()

X_train_3class, Y_train_3class = shuffle(X_train_3class, Y_train_3class)

X_train_3class, X_test_3class, Y_train_3class, Y_test_3class = train_test_split(X_train_3class, Y_train_3class, test_size = 0.3)

X_val_3class = validation_data[feature_cols]
#X_val_3class = X_val_biased
Y_val_3class = validation_data.target

Y_val_3class = Y_val_3class.replace(1, 0)
Y_val_3class = Y_val_3class.replace([0.25, 0.75], 1)
Y_val_3class = Y_val_3class.replace(0.5, 2)

X_val_3class = X_val_3class.to_numpy()
Y_val_3class = Y_val_3class.to_numpy()

In [ ]:
Y_train_3class_oneHot = oneHotEncodeData3Classes(Y_train_3class)
Y_test_3class_oneHot  = oneHotEncodeData3Classes(Y_test_3class)
Y_val_3class_oneHot   = oneHotEncodeData3Classes(Y_val_3class)

test_history = []
val_history = []
class MyCustomCallback_3class(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        res_eval_1 = self.model.evaluate(X_test_3class, Y_test_3class_oneHot, verbose = 0)
        res_eval_2 = self.model.evaluate(X_val_3class, Y_val_3class_oneHot, verbose = 0)
        test_history.append(res_eval_1[0])
        val_history.append(res_eval_2[0])
        print("test ",res_eval_1)
        print("val", res_eval_2)
my_val_callback_3class = MyCustomCallback_3class()

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1, 2]), Y_train_3class)
class_weights = dict(enumerate(class_weights))

test_history = []
val_history = []
history = modelNN_3classes.fit(X_train_3class, Y_train_3class_oneHot, epochs = 100, class_weight=class_weights, batch_size = 128*256, callbacks = [my_val_callback_3class])#validation_data=(X_test_3class, Y_test_3class_oneHot))

In [ ]:
Y_val_3class_oneHot = oneHotEncodeData3Classes(Y_val_3class)

start_date = date(2021,1, 31)
path = "data/numerai_datasets_"
test_date = start_date
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

epochsPerData = 20
epochs = 2

epoch = 0

for epoch in range(epochs):
    start_date = date(2021,1, 31)
    path = "data/numerai_datasets_"
    test_date = start_date
    test_path = path + test_date.strftime("%d.%m.%y")
    delta = timedelta(days=7)
    
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)
        
#         if test_date != date(2021, 4, 11):
#             path_to_csv_val = test_path + "/numerai_validation_data.csv" 
#             if os.path.exists(path_to_csv_val):
#                 val_data = pd.read_csv(str(path_to_csv_val))
#                 val_data_3classes = val_data[training_data.columns]
                
#                 training_data = pd.concat([training_data, val_data_3classes], axis = 0, ignore_index = True)
#         else:
#             break


        X_train_3class = training_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
        Y_train_3class = training_data.target        
        
        Y_train_3class = Y_train_3class.replace(1, 0)
        Y_train_3class = Y_train_3class.replace([0.25, 0.75], 1)
        Y_train_3class = Y_train_3class.replace(0.5, 2)        
        
        Y_train_3class_oneHot = oneHotEncodeData3Classes(Y_train_3class)
        #Y_test_3class_oneHot  = oneHotEncodeData3Classes(Y_test_3class)

        class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1, 2]), Y_train_3class)
        class_weights = dict(enumerate(class_weights))
    
        history = modelNN_3classes.fit(X_train_3class, Y_train_3class_oneHot, epochs = epochsPerData, class_weight=class_weights, batch_size = 256*128, validation_data=(X_val_3class, Y_val_3class_oneHot))

        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
#import pdb; pdb.set_trace()
#model.save_weights("model.h5")

In [ ]:
modelNN_3classes.save_weights("model_3class_100train_92val_noValData.h5")


In [ ]:
pred = modelNN_3classes.predict(X_val_3class)
pred = np.argmax(pred, axis = 1) #convert one hot vectors to labels

accuracy = np.array(Y_val_3class == pred).astype(float).sum() / X_val_3class.shape[0]
print(accuracy)
predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

In [ ]:
pred = modelNN_3classes.predict(X_train_3class)
pred = np.argmax(pred, axis = 1) #convert one hot vectors to labels

accuracy = np.array(Y_train_3class == pred).astype(float).sum() / X_train_3class.shape[0]
print(accuracy)
predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

In [ ]:
predictions_df["target"] = Y_train_3class
predictions_df.target.hist(bins = 10)

# train classifier between 0.25/0.75

In [ ]:
validation_data_025 = validation_data.loc[validation_data.target == 0.25]
validation_data_075 = validation_data.loc[validation_data.target == 0.75]
validation_data_025075 = pd.concat([validation_data_025, validation_data_075], axis = 0, ignore_index = True)


X_val_025075 = validation_data_025075[feature_cols].to_numpy()
Y_val_025075 = validation_data_025075.target
Y_val_025075 = Y_val_025075.replace(0.25, 0)
Y_val_025075 = Y_val_025075.replace(0.75, 1)
Y_val_025075 = Y_val_025075.to_numpy()

# X_val_025075, Y_val_025075 = shuffle(X_val_025075, Y_val_025075)

# training_data_025 = training_data.loc[training_data.target == 0.25]
# training_data_075 = training_data.loc[training_data.target == 0.75]
# training_data_025075 = pd.concat([training_data_025, training_data_075], axis = 0, ignore_index = True)


# X_train_025075 = training_data_025075[feature_cols].to_numpy()
# Y_train_025075 = training_data_025075.target
# Y_train_025075 = Y_train_025075.replace(0.25, 0)
# Y_train_025075 = Y_train_025075.replace(0.75, 1)
# Y_train_025075 = Y_train_025075.to_numpy()

# X_train_025075, Y_train_025075 = shuffle(X_train_025075, Y_train_025075)

# X_train_025075, X_test_025075, Y_train_025075, Y_test_025075 = train_test_split(X_train_025075, Y_train_025075, test_size = 0.3)

In [ ]:
start_date = date(2021,1, 31)
path = "data/numerai_datasets_"
test_date = start_date
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

epochsPerData = 25
epochs = 1
counter = 1
epoch = 0

for epoch in range(epochs):
    
    start_date = date(2021,4, 25)
    path = "data/numerai_datasets_"
    test_date = start_date
    test_path = path + test_date.strftime("%d.%m.%y")
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)
        
        training_data_075 = training_data.loc[training_data.target == 0.75]
        training_data_025 = training_data.loc[training_data.target == 0.25]
        training_data_025075 = pd.concat([training_data_075, training_data_025], axis = 0, ignore_index = True)

#         if test_date != date(2021, 4, 18):
#             path_to_csv_val = test_path + "/numerai_validation_data.csv" 
#             if os.path.exists(path_to_csv_val):
#                 val_data = pd.read_csv(str(path_to_csv_val))
#                 val_data_075 = val_data.loc[val_data.target == 0.75]
#                 val_data_025 = val_data.loc[val_data.target == 0.25]
#                 val_data_025075 = pd.concat([val_data_075, val_data_025], axis = 0, ignore_index = True)
#                 val_data_025075 = val_data_025075[training_data.columns]
                
#                 training_data_025075 = pd.concat([training_data_025075, val_data_025075], axis = 0, ignore_index = True)

        Y_train_025075 = training_data_025075.target
        Y_train_025075 = Y_train_025075.replace(0.25, 0)
        Y_train_025075 = Y_train_025075.replace(0.75, 1)

        training_data_025075["target"] = Y_train_025075

        if counter == 0:
            appendedTraining_data_025075 = training_data_025075
        else:
            appendedTraining_data_025075 = pd.concat([training_data_025075, appendedTraining_data_025075], axis = 0, ignore_index = True)
        
        
        counter += 1

        
                
#         Y_train_025075 = Y_train_025075.to_numpy()
#         X_train_025075 = training_data_025075[feature_cols].to_numpy()
        
#         X_train_025075, Y_train_025075 = shuffle(X_train_025075, Y_train_025075)

#         class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_025075)
#         class_weights = dict(enumerate(class_weights))
        
#         history = modelNN_025075.fit(X_train_025075, Y_train_025075, epochs = epochsPerData, batch_size = 256*128, class_weight=class_weights, validation_data=(X_val_025075, Y_val_025075))
        

        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
        #import pdb; pdb.set_trace()
        #model.save_weights("model.h5")

X_train_025075 = appendedTraining_data_025075[feature_cols].to_numpy()
Y_train_025075 = appendedTraining_data_025075.target.to_numpy()
X_train_025075, Y_train_025075 = shuffle(X_train_025075, Y_train_025075)
        
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_025075)
class_weights = dict(enumerate(class_weights))

In [ ]:
appendedTraining_data_025075.to_csv("appendedTrainingData_classes_025_075_till_02_05_21_noValData.csv")

In [ ]:
training_data_075 = training_data.loc[training_data.target == 0.75]
training_data_025 = training_data.loc[training_data.target == 0.25]
training_data_025075 = pd.concat([training_data_075, training_data_025], axis = 0, ignore_index = True)

Y_train_025075 = training_data_025075.target
Y_train_025075 = Y_train_025075.replace(0.25, 0)
Y_train_025075 = Y_train_025075.replace(0.75, 1)
training_data_025075["target"] = Y_train_025075

# X_train_025075 = appendedTraining_data_025075[feature_cols].to_numpy()
# Y_train_025075 = appendedTraining_data_025075.target.to_numpy()
X_train_025075 = training_data_025075[feature_cols].to_numpy()
Y_train_025075 = training_data_025075.target.to_numpy()
X_train_025075, Y_train_025075 = shuffle(X_train_025075, Y_train_025075)
#X_train_025075, X_test_025075, Y_train_025075, Y_test_025075 = train_test_split(X_train_025075, Y_train_025075, test_size = 0.3)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_025075)
class_weights = dict(enumerate(class_weights))

In [ ]:
modelNN_025075 = neuralNets.defineNN_2classes(X_train_025075.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.99, beta_2=0.99999)
modelNN_025075.compile(optimizer=optAdam, loss='binary_crossentropy', metrics='accuracy')
modelNN_025075.load_weights('model_025075_class_99train_94val_noValData.h5')

In [ ]:
history = modelNN_025075.fit(X_train_025075, Y_train_025075, epochs = 50, batch_size = 256*128, class_weight=class_weights, validation_data=(X_val_025075, Y_val_025075))#(X_test_025075, Y_test_025075)) #

In [ ]:
modelNN_025075.save_weights("model_025075_class_100train_92test_noValData.h5")

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
#print(history.history)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
def max_accuracyNP(target, pred):
    correctVals = 0
    n_examples = target.shape[0]
    trueArray = (np.rint(pred) == np.reshape(target, (n_examples,1))).astype(float)
    wrongValues = 0
    for i in range(len(trueArray)):
        if trueArray[i] == 1:
            correctVals += 1
        elif trueArray[i] == 0:
            wrongValues += 1
        else:
            print("weird value ", trueArray[i])
#     print(trueArray.shape, pred.shape, target.shape)
#     for n in range(len(trueArray)):
#         print(pred[n], target[n])
    print("true", correctVals,"wrong", wrongValues, "n_examples", n_examples)
    return trueArray.sum()/n_examples    

def max_accuracy(target, pred):
  """Calculates how often the max prediction matches one-hot labels."""
  retVal = 0
  #print(pred.shape[0])
  if pred.shape[0] != None:  
    num_correct_classified = (tf.math.round(pred) == target)
    num = tf.reduce_sum(tf.dtypes.cast(num_correct_classified, tf.int32), axis = -1)
    retVal = num / pred.shape[0]
    print(tf.math.round(pred))#, num_currect_classified, retVal)
  else:
    retVal = tf.dtypes.cast(tf.math.round(pred) == target, tf.int32)
  print(retVal)
  return retVal

In [ ]:
pred = modelNN_025075.predict(X_val_025075)

print(max_accuracyNP(Y_val_025075, pred))

predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

# train classifier between 0 and 1

In [ ]:
training_data_1 = training_data.loc[training_data.target == 1]
training_data_0 = training_data.loc[training_data.target == 0]
training_data_01 = pd.concat([training_data_1, training_data_0], axis = 0, ignore_index = True)

X_train_01 = training_data_01[feature_cols].to_numpy()
Y_train_01 = training_data_01.target.to_numpy()
X_train_01, Y_train_01 = shuffle(X_train_01, Y_train_01)

#X_train_01, X_test_01, Y_train_01, Y_test_01 = train_test_split(X_train_01, Y_train_01, test_size = 0.3)

In [ ]:
validation_data_1 = validation_data.loc[validation_data.target == 1]
validation_data_0 = validation_data.loc[validation_data.target == 0]
validation_data_01 = pd.concat([validation_data_1, validation_data_0], axis = 0, ignore_index = True)

X_val_01 = validation_data_01[feature_cols].to_numpy()
Y_val_01 = validation_data_01.target.to_numpy()
#X_val_01, Y_val_01 = shuffle(X_val_01, Y_val_01)

In [ ]:
test_history = []
val_history = []
class MyCustomCallback_01(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        res_eval_1 = self.model.evaluate(X_test_01, Y_test_01, verbose = 0)
        res_eval_2 = self.model.evaluate(X_val_01, Y_val_01, verbose = 0)
        test_history.append(res_eval_1[0])
        val_history.append(res_eval_2[0])
        print("test ",res_eval_1)
        print("val", res_eval_2)
my_val_callback_01 = MyCustomCallback_01()

class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_01)
class_weights = dict(enumerate(class_weights))

In [ ]:
start_date_orig = date(2021,1, 31)
start_date_orig = date(2021,4, 25)
path = "data/numerai_datasets_"
test_date = start_date_orig
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

epochsPerData = 25
epochs = 1
counter = 1
epoch = 0

for epoch in range(epochs):
    
    start_date = start_date_orig
    path = "data/numerai_datasets_"
    test_date = start_date
    test_path = path + test_date.strftime("%d.%m.%y")
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)

        
        training_data_1 = training_data.loc[training_data.target == 1]
        training_data_0 = training_data.loc[training_data.target == 0]
        training_data_01 = pd.concat([training_data_1, training_data_0], axis = 0, ignore_index = True)
        
#         if test_date != date(2021, 4, 11):
#             path_to_csv_val = test_path + "/numerai_validation_data.csv" 
#             if os.path.exists(path_to_csv_val):
#                 val_data = pd.read_csv(str(path_to_csv_val))
#                 val_data_1 = val_data.loc[val_data.target == 1]
#                 val_data_0 = val_data.loc[val_data.target == 0]
#                 val_data_01 = pd.concat([val_data_1, val_data_0], axis = 0, ignore_index = True)
#                 val_data_01 = val_data_01[training_data.columns]
#                 training_data_01 = pd.concat([training_data_01, val_data_01], axis = 0, ignore_index = True)
        
        #append all training data into one dataframe
        if counter == 0:
            appendedTraining_data_01 = training_data_01
        else:
            appendedTraining_data_01 = pd.concat([training_data_01, appendedTraining_data_01], axis = 0, ignore_index = True)
        
#         X_train_01 = training_data_01[feature_cols].to_numpy()
#         Y_train_01 = training_data_01.target.to_numpy()
#         X_train_01, Y_train_01 = shuffle(X_train_01, Y_train_01)
        
#         class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_01)
#         class_weights = dict(enumerate(class_weights))


    
#        history = modelNN_01.fit(X_train_01, Y_train_01, epochs = epochsPerData, batch_size = 256*128*2, class_weight=class_weights, validation_data=(X_test_01, Y_test_01))

        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
        counter += 1
        #import pdb; pdb.set_trace()
        #model.save_weights("model.h5")
 

# train with one appended matrix
print(appendedTraining_data_01.shape)
X_train_01 = appendedTraining_data_01[feature_cols].to_numpy()
Y_train_01 = appendedTraining_data_01.target.to_numpy()
X_train_01, Y_train_01 = shuffle(X_train_01, Y_train_01)
        
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_01)
class_weights = dict(enumerate(class_weights))


In [ ]:
modelNN_01 = neuralNets.defineNN_2classes(X_train_01.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.99)
modelNN_01.compile(optimizer=optAdam, loss='binary_crossentropy', metrics='accuracy')
modelNN_01.load_weights('model_01class_100train_96val_noValData.h5')

In [ ]:
history = modelNN_01.fit(X_train_01, Y_train_01, epochs = 100, batch_size = 256*128*2, class_weight=class_weights, validation_data=(X_val_01, Y_val_01))


In [ ]:
appendedTraining_data_01.to_csv("appendedTrainingData_classes_0_1_till_18_04_21_noValData.csv")

In [ ]:
modelNN_01.save_weights("model_01class_100train_95val_noValData.h5")